In [1]:
# The usuals
import os
import sys
import subprocess
import pandas as pd
import numpy as np

# Useful stuff
from zipfile import ZipFile
from joblib import dump, load
from pprint import pprint
from pandas_profiling import ProfileReport
from IPython.display import IFrame
import random
from statistics import mode
from datetime import datetime
from scipy.stats import uniform, randint

# SkLearn bits
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# PyTorch bits
import torch
from torch import nn
from torch.nn import functional as F

# Autoreload
%load_ext autoreload
%autoreload

In [2]:
# https://drive.google.com/u/0/uc?export=download&confirm=IpWH&id=1vYyJL_IB6KjKCxuk9kg4vIMPGTtoX8Ek

In [3]:
# Ensure the directory is correct... every time.
for i in range(5):
    if not os.getcwd().lower() == subprocess.run("git rev-parse --show-toplevel", stdout=subprocess.PIPE).stdout.decode("utf-8").replace("/","\\").strip().lower():
        os.chdir(".."),
    else:
        break

In [4]:
# Set up sys path environment
if not os.path.abspath(".") in sys.path:
    sys.path.append(os.path.abspath("."))
else:
    sys.path.remove(os.path.abspath("."))
    sys.path.append(os.path.abspath("."))

In [5]:
print(os.getcwd())

c:\Users\chris\OneDrive\02 - Education\07 - MDSI\09 - ADSI\07 - GitHub Repo\BeerPrediction


In [18]:
data = pd.read_csv("./data/raw/beer_reviews.csv")

In [19]:
print(data.shape)
print(data.columns)
display(data.describe())

(1586614, 13)
Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv',
       'beer_beerid'],
      dtype='object')


,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06,1.586614e+06
mean,3.130099e+03,1.224089e+09,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00,2.171279e+04
std,5.578104e+03,7.654427e+07,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00,2.181834e+04
min,1.000000e+00,8.406720e+08,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,3.000000e+00
25%,1.430000e+02,1.173224e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.717000e+03
50%,4.290000e+02,1.239203e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.390600e+04
75%,2.372000e+03,1.288568e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.944100e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731700e+04


In [20]:
# from pandas_profiling import ProfileReport

# # Create profile report
# profile = ProfileReport(data, title="Profile Report")
# # Export
# profile.to_file("./reports/InitialReport.html")
# # View
# display(profile)


In [21]:
data_upd = data.copy()

In [22]:
print(data_upd.columns)
from src.data.set_data import rem_features
data_upd = rem_features(data_upd, ["brewery_id","beer_beerid","beer_name","beer_abv","beer_beerid"])
print(data_upd.columns)

Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv',
       'beer_beerid'],
      dtype='object')
Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv',
       'beer_beerid'],
      dtype='object')
Index(['brewery_name', 'review_time', 'review_overall', 'review_aroma',
       'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste'],
      dtype='object')


In [23]:
from src.data.set_data import pop_target

feat, targ = pop_target(data_upd, "beer_style")

print(targ.shape)
print(feat.shape)

(1586614,)
(1586614, 8)


In [24]:
from src.data.set_data import split_data
from src.utils.misc import get_shape
feat_trn, feat_tst, targ_trn, targ_tst = split_data(feat, targ, test_size=0.3)
feat_trn, feat_val, targ_trn, targ_val = split_data(feat_trn, targ_trn, test_size=0.3)

[print(data, ": ", get_shape(eval(data), True)) for data in ["feat_trn", "targ_trn", "feat_val", "targ_val", "feat_tst", "targ_tst"]]

feat_trn :  777440 8
targ_trn :  777440
feat_val :  333189 8
targ_val :  333189
feat_tst :  475985 8
targ_tst :  475985


[None, None, None, None, None, None]